## Code build and train ANN

In [ ]:
include("./code/general/utils_general.jl")

include("./code/ann/build_train.jl")
include("./code/ann/utils_ann.jl")

using Pkg; Pkg.add("DelimitedFiles");    
using DelimitedFiles

using Pkg; Pkg.add("Plots")
using Plots; Pkg.add("PyPlot")



#Load iris dataset
dataset = readdlm("./data_test/iris.data",',');
datasetLength = size(dataset, 1)


(trainIndexes, validationIndexes, testIndexes) = holdOut(datasetLength, 0.2, 0.2)
(trainDataset, validationDataset, testDataset) = (dataset[trainIndexes, :], dataset[validationIndexes, :], dataset[testIndexes, :])


(trainInput, validationInput, testInput) = (convert(Array{Float32,2}, trainDataset[:,1:4]), convert(Array{Float32,2}, validationDataset[:,1:4]), convert(Array{Float32,2}, testDataset[:,1:4]))


classes = unique(trainDataset[:,5])
(trainOutput, validationOutput, testOutput) = (oneHotEncoding(trainDataset[:,5], classes), oneHotEncoding(validationDataset[:,5], classes), oneHotEncoding(testDataset[:,5], classes))


# Normalization 
normalizationParameters = calculateMinMaxNormalizationParameters(trainInput);
(trainInputNormalized, validationInputNormalized, testInputNormalized) = (normalizeMinMax(trainInput, normalizationParameters), normalizeMinMax(validationInput, normalizationParameters), normalizeMinMax(testInput, normalizationParameters))


#Define the parameters
topology = [4, 3]; 
learningRate = 0.01;
numMaxEpochs = 1000; 
topologies_to_try = [[5], [10], [4, 3], [8, 4]]
    
for topo in topologies_to_try
    println("Training with topology: ", topo)
    (ann, trainingLosses, validationLosses, testLosses) = trainClassANN(topo, (trainInputNormalized, trainOutput); validationDataset=(validationInputNormalized, validationOutput), testDataset=(testInputNormalized, testOutput), maxEpochs=numMaxEpochs, learningRate=learningRate);

  g =  plot(1:length(trainingLosses), 
    trainingLosses, 
    label="Train losses", 
    xlabel="Epoch", 
    ylabel="Loss value", 
    title="Loss evolution")

    if !isempty(validationLosses)
        plot!(1:length(validationLosses), validationLosses, label="Validation losses")
    end

    if !isempty(testLosses)
        plot!(1:length(testLosses), testLosses, label="Test losses")
    end

    display(g)
end;  


## Code metrics

In [9]:
include("./code/general/train_metrics.jl")

#Test boolean and boolean function
outputs=[0,0,1,1,0,1,1,0,0,0,1,1]
targets=[0,1,1,0,1,1,1,0,1,1,0,0]
printConfusionMatrix(convert(Array{Bool,1}, outputs), convert(Array{Bool,1}, targets));
println("\n\n")

#Test real and boolean
outputs=[0.4; 0.6; 1; 0.4;  0.55]
targets=[0;   1;   1; 1; 0]
printConfusionMatrix(convert(Array{Real,1}, outputs), convert(Array{Bool,1}, targets));


#Test array bool
testOutputs=Bool[0 0 1;1 0 0;0 1 0; 0 0 1; 0 0 1]
testTargets=Bool[0 0 1;0 1 0;0 1 0; 1 0 0; 1 0 0]
printConfusionMatrix(testOutputs, testTargets, weighted=false)
println("\n\n")

#Test array real and target bool
testRealOutputs=Real[0 0 0.7;0 0 0.6;0 1 0; 0 0 1; 0 0 1]
printConfusionMatrix(testRealOutputs, testTargets, weighted=false)
println("\n\n")

#Test array any 
result = confusionMatrix(testOutputs, testTargets)

testClasses=["dog", "cat", 3, :green, "dog", 3, "dog"]
testClassesTargets=["dog", "cat", 3, :green, "dog", 3, "dog"]
uniqueClasss=unique(testClasses)
printConfusionMatrix(testClasses, testClassesTargets, uniqueClasss)
println("\n\n")

#Test two classes 
result = confusionMatrix(testOutputs, testTargets)

testClasses=["dog"; "cat"]
testClassesTargets=["dog"; "cat"]
uniqueClasss=unique(testClasses)
printConfusionMatrix(testClasses, testClassesTargets, uniqueClasss)
println("\n\n")
println("hola")

## BUILD AND TRAIN CROSS VALIDATION ANN

In [7]:
using Pkg; Pkg.add("DelimitedFiles");    
using DelimitedFiles

include("./code/general/utils_general.jl")
include("./code/ann/build_train.jl")
include("./code/ann/utils_ann.jl")


a=[false,true, false, true, true, false]


result = crossvalidation(5, 3)

println("Result", result)

b = crossvalidation(a, 3)
println("Cross validation AbstractArray{Bool, 1}", b)


a = Bool[0 1;1 0;1 0; 0 1;0 1]

bool2_indices = crossvalidation(a, 10)
println("Cross validation AbstractArray{Bool,2} ", bool2_indices)

unique(bool2_indices)


any_array = ['a', :b, 1, 'a']
one = oneHotEncoding(any_array)

println("Size ", size(any_array, 1), " lenght one " ,length(one), " size one ", size(one), "number columns one", size(one, 2))

any_indices = crossvalidation(any_array, 2)
println("Cross validation AbstractArray{<:Any,2} ", any_indices)


#HOW TO

b = [1 1 1 "perro"; 0 0 0 "gato"]


#Load iris dataset
dataset = readdlm("./data_test/iris.data",',');
datasetLength = size(dataset, 1)


(trainIndexes, testIndexes) = holdOut(datasetLength, 0.2)
(trainDataset, testDataset) = (dataset[trainIndexes, :], dataset[testIndexes, :])
(trainInput, testInput) = (convert(Array{Float32,2}, trainDataset[:,1:4]), convert(Array{Float32,2}, testDataset[:,1:4]))



classes = unique(trainDataset[:,5])
normalizationParameters = calculateMinMaxNormalizationParameters(trainInput);
(trainInputNormalized, testInputNormalized) = (normalizeMinMax(trainInput, normalizationParameters), normalizeMinMax(testInput, normalizationParameters))

topology = [4, 3]; 
crossValidationIndices = crossvalidation(trainDataset[:, 5], 8)


ANNCrossValidation(topology,
        (trainInputNormalized, trainDataset[:, 5]),
        crossValidationIndices)

   Resolving package versions...


  No Changes to `C:\Users\Pablo\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Pablo\.julia\environments\v1.11\Manifest.toml`


   Resolving package versions...


  No Changes

 to `C:\Users\Pablo\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\Pablo\.julia\environments\v1.11\Manifest.toml`


LoadError: LoadError: SystemError: opening file "c:\\Users\\Pablo\\Documents\\Master Inteligencia Artifical\\Aprendizaje Automatico\\Laboratoy\\Proyecto final\\code\\general\\utils_metrics.jl": No such file or directory
in expression starting at c:\Users\Pablo\Documents\Master Inteligencia Artifical\Aprendizaje Automatico\Laboratoy\Proyecto final\code\ann\build_train.jl:6